In [33]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:,.0f}'.format

In [5]:
path = 'datasets/employee_list.parquet'
df = pd.read_parquet(path)
df.head()

,employee_number,name,company,country,dob,age,department,salary,has_parking_space
0,897028,Kenneth Jensen,Wilson and Sons,India,1983-07-03,38,Management,124790,False
1,463979,Sarah Anderson,"Hernandez, Cunningham and Clark",India,1980-08-09,41,Consulting,103122,True
2,388446,Tracie Rollins,"Hernandez, Cunningham and Clark",Cayman Islands,1987-07-29,34,Consulting,119072,False
3,267447,Seth Smith,Spears-Brown,Germany,1969-03-04,52,System Architect,115653,False
4,401300,Katherine Fields,"Hernandez, Cunningham and Clark",Venezuela,1980-01-26,42,Finance,119412,False


## Pivot table

**Different aggfunc methods:**
- `count` - The number of group values
- `sum` / `np.sum` - The sum of group values
- `mean` / `np.mean`    -   The average of a group values
- `median` -  The middle of the group values
- `np.std`
- `max`
- `min`
- `sem` -  Standard error of the mean
- `quantile`
- `nunique`
- `mad`
- `size`
- `mode` / `pd.Series.mode` - The most common number in a group
- `var`
- `unique` - Unique group values

##### Mean salary within each department of each company

In [10]:
df.pivot_table(index='company', columns='department', values='salary', aggfunc='mean')

department,Consulting,Developer,Finance,Management,System Architect
company,,,,,
"Hernandez, Cunningham and Clark","108,412","101,506","104,112","98,756","108,414"
Spears-Brown,"108,118","102,611","107,777","120,032","103,507"
Wilson and Sons,"124,710","97,369","89,296","115,703","110,295"


##### Number of employees within each department of each company

In [11]:
df.pivot_table(index='company', columns='department', values='name', aggfunc='count')

department,Consulting,Developer,Finance,Management,System Architect
company,,,,,
"Hernandez, Cunningham and Clark",7,4,9,7,9
Spears-Brown,3,7,7,8,8
Wilson and Sons,6,7,3,7,8


##### Oldest age within each department of each company

In [16]:
df.pivot_table(index='company', columns='department', values='age', aggfunc='max')

department,Consulting,Developer,Finance,Management,System Architect
company,,,,,
"Hernandez, Cunningham and Clark",50,48,53,49,53
Spears-Brown,53,50,53,51,53
Wilson and Sons,49,52,45,51,53


##### Longest name within each department of each company

In [18]:
df.pivot_table(index='company', columns='department', values='name', aggfunc='size')

department,Consulting,Developer,Finance,Management,System Architect
company,,,,,
"Hernandez, Cunningham and Clark",7,4,9,7,9
Spears-Brown,3,7,7,8,8
Wilson and Sons,6,7,3,7,8


#### Pivot table containing oldest employee and his/her age for each company and department

In [23]:
(
df
.sort_values('dob')
.drop_duplicates(
    subset=['company', 'department'], 
    ascending=True)
.pivot_table(
    index='company', 
    columns='department', 
    values=['name', 'age'], 
    aggfunc='first')
)

age                               \
department                      Consulting Developer Finance Management   
company                                                                   
Hernandez, Cunningham and Clark         50        48      53         49   
Spears-Brown                            53        50      53         51   
Wilson and Sons                         49        52      45         51   

                                                               name  \
department                      System Architect         Consulting   
company                                                               
Hernandez, Cunningham and Clark               53  Kimberly Williams   
Spears-Brown                                  53     Bruce Crawford   
Wilson and Sons                               53       Robert Carey   

                                                                     \
department                             Developer            Finance   
company                                                               
Hernandez, Cunningham and Clark     Logan Garcia      Andrew Rivera   
Spears-Brown                     Justin Anderson         Linda Page   
Wilson and Sons                   Jonathan Brown  Andrew Strickland   

                                                                     
department                              Management System Architect  
company                                                              
Hernandez, Cunningham and Clark  Zachary Gallagher       Susan Horn  
Spears-Brown                        Sophia Wallace  Benjamin Snyder  
Wilson and Sons                     Larry Robinson      Julie Craig

## Melt
- Turns the dataframe from wide (many columns) to tall (many rows)

In [36]:
# Mean salary within each department of each company
piv = df.pivot_table(index='company', columns='department', values='salary', aggfunc='mean').reset_index()
piv

department,company,Consulting,Developer,Finance,Management,System Architect
0,"Hernandez, Cunningham and Clark","108,412","101,506","104,112","98,756","108,414"
1,Spears-Brown,"108,118","102,611","107,777","120,032","103,507"
2,Wilson and Sons,"124,710","97,369","89,296","115,703","110,295"


In [46]:
piv.melt(id_vars='company').sort_values('company')

,company,department,value
0,"Hernandez, Cunningham and Clark",Consulting,"108,412"
3,"Hernandez, Cunningham and Clark",Developer,"101,506"
6,"Hernandez, Cunningham and Clark",Finance,"104,112"
9,"Hernandez, Cunningham and Clark",Management,"98,756"
12,"Hernandez, Cunningham and Clark",System Architect,"108,414"
1,Spears-Brown,Consulting,"108,118"
4,Spears-Brown,Developer,"102,611"
7,Spears-Brown,Finance,"107,777"
10,Spears-Brown,Management,"120,032"
13,Spears-Brown,System Architect,"103,507"


In [49]:
(piv
.melt(
    id_vars='company', 
    # Only include data for dvelopers and system akjkrchitects
    value_vars=['Developer', 'System Architect'], 
    value_name='mean_salary')
.sort_values('company'))

,company,department,mean_salary
0,"Hernandez, Cunningham and Clark",Developer,"101,506"
3,"Hernandez, Cunningham and Clark",System Architect,"108,414"
1,Spears-Brown,Developer,"102,611"
4,Spears-Brown,System Architect,"103,507"
2,Wilson and Sons,Developer,"97,369"
5,Wilson and Sons,System Architect,"110,295"
